<a href="https://colab.research.google.com/github/DATA3750/WeeklyDemo/blob/main/Wk7_Python_PowerBI_missingdata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Missing Data

## Outline

___________
* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)
* [Export the cleaned dataset to a CSV file](#Export-the-cleaned-dataset-to-a-CSV-file) 

## Take a first look at the data
________

We are using a dataset of [building permits issued in San Francisco](https://data.sfgov.org/Housing-and-Buildings/Building-Permits/i98e-djp9/data).

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# read in all our data
permits = pd.read_csv("Building_Permits.csv")

# set seed for reproducibility
# Setting the seed for the random number generator ensures that the same sequence of random numbers 
# will be generated every time the code is run.
# This can be useful for testing and debugging purposes, 
# as it allows you to reproduce the same results every time you run the code.
np.random.seed(0) 

C:\Users\DONGY\AppData\Local\Temp\ipykernel_23736\402575328.py:7: DtypeWarning: Columns (18,22) have mixed types. Specify dtype option on import or set low_memory=False.
  permits = pd.read_csv("Building_Permits.csv")


Explore the data to see if there are any missing values, which will be reprsented with `NaN` or `None`.

In [ ]:
# look at a few rows of the nfl_data file.
permits.sample(5)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
43584,202209273207,8,otc alterations permit,09/27/2022,0289,004,1,NaN,Sansome,St,...,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Financial District/South Beach,94104.0,POINT (-122.40135901183898 37.79043181182496),165260067086
26366,202201286861,8,otc alterations permit,01/28/2022,3707,052,2,NaN,New Montgomery,St,...,1.0,constr type 1,1.0,constr type 1,NaN,6.0,Financial District/South Beach,94105.0,POINT (-122.40193430307012 37.78794073815317),1634685164500
17648,202109178619,8,otc alterations permit,09/17/2021,4277,013,2789,NaN,25th,St,...,1.0,constr type 1,1.0,constr type 1,NaN,10.0,Mission,94110.0,POINT (-122.40566262423741 37.75145965266103),1625602173128
47144,202211156572,8,otc alterations permit,11/15/2022,3708,096,55,NaN,02nd,St,...,1.0,constr type 1,1.0,constr type 1,NaN,6.0,Financial District/South Beach,94121.0,POINT (-122.40031877127254 37.78885628497609),1656271394477
53066,202302202258,8,otc alterations permit,02/20/2023,7009,040,149,NaN,Garfield,St,...,NaN,NaN,NaN,NaN,NaN,11.0,Oceanview/Merced/Ingleside,94132.0,POINT (-122.46430977554711 37.719491032198476),1662446217439


## How many missing data points?
___

In [ ]:
# get the number of missing data points per column
missing_values_count = permits.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count

Permit Number                                 0
Permit Type                                   0
Permit Type Definition                        0
Permit Creation Date                          0
Block                                         0
Lot                                           0
Street Number                                 0
Street Number Suffix                      52704
Street Name                                   0
Street Suffix                               743
Unit                                      46301
Unit Suffix                               52738
Description                                   5
Current Status                                0
Current Status Date                           0
Filed Date                                    0
Issued Date                                7734
Completed Date                            28012
First Construction Document Date          53360
Structural Notification                   51540
Number of Existing Stories              

That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

In [ ]:
# how many total missing values do we have?
total_cells = np.product(permits.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

24.384617393120735

Almost a quarter of the cells in this dataset are empty! 

## Why is the data missing?
____

For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. 

On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation".)


Looking at the number of missing values in the permits dataframe, we can see that the columns `Street Number Suffix` and `zipcode` have a lot of missing values.

In [ ]:
# look at the # of missing points.
missing_values_count[7:]

Street Number Suffix                      52704
Street Name                                   0
Street Suffix                               743
Unit                                      46301
Unit Suffix                               52738
Description                                   5
Current Status                                0
Current Status Date                           0
Filed Date                                    0
Issued Date                                7734
Completed Date                            28012
First Construction Document Date          53360
Structural Notification                   51540
Number of Existing Stories                 3870
Number of Proposed Stories                 4070
Voluntary Soft-Story Retrofit             53436
Fire Only Permit                          46203
Permit Expiration Date                     8029
Estimated Cost                              559
Revised Cost                               2553
Existing Use                            

This is the time to dig into the [the documentation](https://data.sfgov.org/Housing-and-Buildings/Building-Permits/i98e-djp9/data). [Data dictionary](https://data.sfgov.org/api/views/i98e-djp9/files/0bff3a42-53fd-4aaa-b99f-32a289bd8aa6?download=true&filename=DBI-0002_DataDictionary-building-permits.xlsx) is useful, too.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values.

* Look at the columns `Street Number Suffix` and `Zipcode` from the `permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

## Drop missing values?
___

If you could not figure out why the values are missing, one option is to just remove any rows or columns that contain missing values. 

> **Note**: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. 

In [ ]:
# remove all the rows that contain a missing value
permits.dropna()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID


And we removed all our data! 😱 This is because every row in thr dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

In [ ]:
# remove all columns with at least one missing value
columns_with_na_dropped = permits.dropna(axis=1)
columns_with_na_dropped.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,202101042027,3,additions alterations or repairs,01/04/2021,4160,059,1033,De Haro,filed,01/04/2021,01/04/2021,1607364171658
1,202101042035,3,additions alterations or repairs,01/04/2021,5991,002,302,University,filed,01/04/2021,01/04/2021,1607371492478
2,202101042041,3,additions alterations or repairs,01/04/2021,5991,004,308,University,filed,01/04/2021,01/04/2021,1607379492477
3,202101042033,3,additions alterations or repairs,01/04/2021,5991,001,300,University,filed,01/04/2021,01/04/2021,1607369492475
4,202101042037,3,additions alterations or repairs,01/04/2021,5991,003,306,University,filed,01/04/2021,01/04/2021,1607375492476


In [ ]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % permits.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 43 

Columns with na's dropped: 12


We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

## Filling in missing values automatically?
_____

Another option is to try and fill in the missing values.

### `fillna()` Method

In [ ]:
# get a small subset of the NFL dataset
subset_permits = permits.loc[:, 'Permit Type':'Street Name'].head()
subset_permits

,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name
0,3,additions alterations or repairs,01/04/2021,4160,059,1033,NaN,De Haro
1,3,additions alterations or repairs,01/04/2021,5991,002,302,NaN,University
2,3,additions alterations or repairs,01/04/2021,5991,004,308,NaN,University
3,3,additions alterations or repairs,01/04/2021,5991,001,300,NaN,University
4,3,additions alterations or repairs,01/04/2021,5991,003,306,NaN,University


We can use the Panda's fillna() function to fill in missing values in a dataframe. I would like to replace all the `NaN` values with 0.

In [ ]:
# replace all NA's with 0
subset_permits.fillna(0)

,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name
0,3,additions alterations or repairs,01/04/2021,4160,059,1033,0,De Haro
1,3,additions alterations or repairs,01/04/2021,5991,002,302,0,University
2,3,additions alterations or repairs,01/04/2021,5991,004,308,0,University
3,3,additions alterations or repairs,01/04/2021,5991,001,300,0,University
4,3,additions alterations or repairs,01/04/2021,5991,003,306,0,University


Alternatively, we can replace all the NaN's in the `permits` data with the one that comes directly after it in the same column and then replacing any remaining NaN's with 0

In [ ]:
# Create a dataframe with some missing values
data = {'A': [1, 2, 3, None, 5],
        'B': [6, None, 8, None, 10],
        'C': [11, 12, 13, 14, None]}
data = pd.DataFrame(data)

# Fill missing values in permits with backfilling and 0
data = data.fillna(method='bfill', axis=0).fillna(0)
print(data)


     A     B     C
0  1.0   6.0  11.0
1  2.0   8.0  12.0
2  3.0   8.0  13.0
3  5.0  10.0  14.0
4  5.0  10.0   0.0


In [ ]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
permits.fillna(method = 'bfill', axis=0).fillna(0)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,2.02101E+11,3,additions alterations or repairs,1/4/2021,4160,59,1033,A,De Haro,St,...,5.0,wood frame (5),5.0,wood frame (5),Y,10.0,Potrero Hill,94107.0,POINT (-122.40050776741901 37.756664350280005),1.607360e+12
1,2.02101E+11,3,additions alterations or repairs,1/4/2021,5991,2,302,A,University,St,...,5.0,wood frame (5),5.0,wood frame (5),Y,9.0,Portola,94134.0,POINT (-122.41423325593475 37.72627924781995),1.607370e+12
2,2.02101E+11,3,additions alterations or repairs,1/4/2021,5991,4,308,A,University,St,...,5.0,wood frame (5),5.0,wood frame (5),Y,9.0,Portola,94134.0,POINT (-122.41417938708821 37.72614876568552),1.607380e+12
3,2.02101E+11,3,additions alterations or repairs,1/4/2021,5991,1,300,A,University,St,...,5.0,wood frame (5),5.0,wood frame (5),Y,9.0,Portola,94134.0,POINT (-122.41426019000068 37.72634448912051),1.607370e+12
4,2.02101E+11,3,additions alterations or repairs,1/4/2021,5991,3,306,A,University,St,...,5.0,wood frame (5),5.0,wood frame (5),Y,9.0,Portola,94134.0,POINT (-122.41420632397862 37.72621400653094),1.607380e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53435,2.02302E+11,8,otc alterations permit,2/24/2023,3516,3,216,0,11th,St,...,3.0,constr type 3,3.0,constr type 3,0,6.0,Mission,94103.0,POINT (-122.41548520282338 37.77268635931323),1.662800e+12
53436,2.02302E+11,8,otc alterations permit,2/24/2023,259,26,555,0,California,St,...,1.0,constr type 1,1.0,constr type 1,0,3.0,Financial District/South Beach,94104.0,POINT (-122.4034859548936 37.79226164705224),1.662750e+12
53437,2.02302E+11,8,otc alterations permit,2/24/2023,3508,39,875,0,Stevenson,St,...,1.0,constr type 1,1.0,constr type 1,0,6.0,South of Market,94122.0,POINT (-122.4162428787466 37.776293204565604),1.662750e+12
53438,2.02302E+11,8,otc alterations permit,2/24/2023,115,52,421,0,Union,St,...,5.0,wood frame (5),5.0,wood frame (5),0,3.0,North Beach,94122.0,POINT (-122.40629941002422 37.800596743383394),1.662820e+12


### Imputation

Imputation is a technique used to handle missing data in a dataset by replacing missing values with estimated values. There are several methods of imputation, including mean imputation, median imputation, and mode imputation, among others. 

In [ ]:
# Create an imputer object with mean strategy
imputer = SimpleImputer(strategy='mean')

# Fit the imputer to the "estimated cost" column
imputer.fit(permits[['Estimated Cost']])

# Transform the missing values in the "estimated cost" column
permits['Estimated Cost'] = imputer.transform(permits[['Estimated Cost']])

In [ ]:
# Check for missing data in the column
print(permits['Estimated Cost'].isnull().sum())

0


### Zipcode

In [ ]:
print(permits['Location'].isnull().sum())

391


In [ ]:
print(permits['Zipcode'].isnull().sum())

608


In [ ]:
# Fill missing values in location column with default value
default_location = 'POINT(-122.431297 37.773972)'
permits['Location'] = permits['Location'].fillna(default_location)

In [ ]:
print(permits['Location'].isnull().sum())

0


In [ ]:
# Convert the Location column to separate columns for latitude and longitude
permits[['Longitude', 'Latitude']] = permits['Location'].str.replace('POINT\(', '').str.replace('\)', '').str.replace('\(', '').str.split(expand=True)
permits['Longitude'] = permits['Longitude'].astype(float)
permits['Latitude'] = permits['Latitude'].astype(float)

In [ ]:
permits_subset = permits[['Location', 'Longitude', 'Latitude']]
print(permits_subset.head())

                                   Location            Longitude  \
0   (-122.40050776741901 37.756664350280005  -122.40050776741901   
1    (-122.41423325593475 37.72627924781995  -122.41423325593475   
2    (-122.41417938708821 37.72614876568552  -122.41417938708821   
3    (-122.41426019000068 37.72634448912051  -122.41426019000068   
4    (-122.41420632397862 37.72621400653094  -122.41420632397862   

             Latitude  
0  37.756664350280005  
1   37.72627924781995  
2   37.72614876568552  
3   37.72634448912051  
4   37.72621400653094  


In [ ]:
# Drop the original location column
permits.drop('Location', axis=1, inplace=True)

In [ ]:
permits['Longitude'] = permits['Longitude'].astype(float)
permits['Latitude'] = permits['Latitude'].astype(float)

In [ ]:
!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim

# Create a geocoder object
geolocator = Nominatim(user_agent='my-app')

# Define a function to get the zip code from latitude and longitude
def get_zipcode(latitude, longitude):
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    address = location.raw['address']
    zipcode = address.get('postcode', '')
    return zipcode

# Fill in missing zip codes
for index, row in permits.iterrows():
    if pd.isna(row['Zipcode']):
        zipcode = get_zipcode(row['Latitude'], row['Longitude'])
        permits.at[index, 'Zipcode'] = zipcode


we use the Nominatim geocoder from the geopy library to perform reverse geocoding. We define a function get_zipcode that takes latitude and longitude as inputs, calls the reverse method of the geolocator object to obtain a location object, extracts the zip code from the address dictionary of the location, and returns the zip code as a string.

We then iterate over each row of the permits DataFrame using the iterrows method, and for each row with a missing Zipcode value, we call the get_zipcode function with the Latitude and Longitude values of the row, and update the Zipcode value of the row with the returned zip code using the at method of the DataFrame.

>**Note** that using a reverse geocoding service may have limitations on the number of requests you can make per day or per IP address, and may also be subject to rate limiting or usage fees. It is important to check the terms of service of the service you are using and comply with any usage limits or guidelines.

In [ ]:
print(permits['Zipcode'].isnull().sum())

0


## Export the cleaned dataset to a CSV file
___

In [ ]:
# Save the updated dataframe
permits.to_csv('permits_updated.csv', index=False)